In [ ]:
# plant stress detection

# Libraries
import json
import datetime
from datetime import datetime, timedelta
from matplotlib.patches import Patch
import os
import rasterio
from rasterio.mask import mask as rio_mask
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import Polygon
import shutil
# import sentinelhub
from sentinelhub import SHConfig
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

def plantstress(service_name,crop_name,org_name,analysis_date,polygon_c,sowing_date):
    gdal.PushErrorHandler('CPLQuietErrorHandler')

    with open("/content/drive/MyDrive/internship/B - with GDAL/python_services_config.json","r")as json_file:
       get_configuration_potato=json.load(json_file)

    config_data_plant_stress=get_configuration_potato['potato_plant_stress']
    config_data_main=get_configuration_potato['main']

    config = SHConfig()

    config.sh_client_id = config_data_main["sh_client_id"]
    config.sh_client_secret = config_data_main["sh_client_secret"]
    config.save()

    today=datetime.now()
    save_date=today.strftime("%Y-%m-%d-%H-%m-%s")

    # output folder
    home_directory = os.path.expanduser("~")
    cc_output_directory = os.path.join(home_directory,config_data_plant_stress['output_folder'])
    os.makedirs(cc_output_directory, exist_ok=True)


    #extracting the date
    def extract_start_date(analysis_date):
        e=datetime.strptime(analysis_date,'%Y-%m-%d')
        s=e-timedelta(7)
        start_date=s.strftime('%Y-%m-%d')
        return start_date

    start_date=extract_start_date(analysis_date)
    end_date=analysis_date


    #shapefile path

    min_x = min(p[0] for p in polygon_c)
    max_x = max(p[0] for p in polygon_c)
    min_y = min(p[1] for p in polygon_c)
    max_y = max(p[1] for p in polygon_c)


    bounding_box = (min_x, min_y, max_x, max_y)
    # print(bounding_box)
    resolution = 10
    betsiboka_bbox = BBox(bbox=bounding_box, crs=CRS.WGS84)
    betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

    # download all the bands
    evalscript_all_bands = """
            //VERSION=3
            function setup() {
                return {
                    input: [{
                        bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                        units: "DN"
                    }],
                    output: {
                        bands: 13,
                        sampleType: "INT16"
                    }
                };
            }

            function evaluatePixel(sample) {
                return [sample.B01,
                        sample.B02,
                        sample.B03,
                        sample.B04,
                        sample.B05,
                        sample.B06,
                        sample.B07,
                        sample.B08,
                        sample.B8A,
                        sample.B09,
                        sample.B10,
                        sample.B11,
                        sample.B12];
            }
        """

    request_all_bands = SentinelHubRequest(
        evalscript=evalscript_all_bands,
        data_folder=config_data_main["data_output_folder"],
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=(start_date, end_date),
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )

    all_bands_response = request_all_bands.get_data()

    # Clear the contents of the data folder before making a new request

    all_bands_img = request_all_bands.get_data(save_data=True)

    for folder, _, filenames in os.walk(request_all_bands.data_folder):
        for filename in filenames:
            file_path = os.path.join(folder, filename)
            # print('d')
            if filename.endswith(".tif") or filename.endswith(".tiff"):

                try:
                    # print("main")
                    def clip_and_remove_zero_pixels(file_a,path_b,polygon_c):
                        # Clip the raster using the provided polygon
                        with rasterio.open(file_a) as src:
                            # Convert the coordinates to a Shapely Polygon
                            polygon = Polygon(polygon_c)

                            # Clip the raster using the polygon
                            out_image, out_transform = rio_mask(src, [polygon], crop=True)

                            # Update the metadata
                            out_meta = src.meta.copy()
                            out_meta.update({"driver": "GTiff",
                                             "height": out_image.shape[1],
                                             "width": out_image.shape[2],
                                             "transform": out_transform})

                        # Create output raster after clipping
                        clipped_raster_path = "file.tiff"
                        with rasterio.open(clipped_raster_path, "w", **out_meta) as dest:
                            dest.write(out_image)

                        # Remove zero pixels from the clipped raster
                        src_ds = gdal.Open(clipped_raster_path)
                        if src_ds is None:
                            print("Failed to open the clipped raster file.")
                            return

                        # Get raster dimensions
                        cols = src_ds.RasterXSize
                        rows = src_ds.RasterYSize
                        bands = src_ds.RasterCount

                        # Create output raster
                        driver = gdal.GetDriverByName("GTiff")
                        dst_ds = driver.Create(path_b, cols, rows, bands, gdal.GDT_Float32)

                        # Set projection and geotransform
                        dst_ds.SetProjection(src_ds.GetProjection())
                        dst_ds.SetGeoTransform(src_ds.GetGeoTransform())

                        # Process each band
                        for band_idx in range(1, bands + 1):
                            # Read band data
                            band = src_ds.GetRasterBand(band_idx)
                            band_array = band.ReadAsArray()

                            # Create a mask for pixels with zero value
                            mask_array = (band_array == -99999)

                            # Apply the mask to the band array
                            masked_band_array = np.where(mask_array, np.nan, band_array)

                            # Write processed band to output raster
                            dst_band = dst_ds.GetRasterBand(band_idx)
                            dst_band.WriteArray(masked_band_array)

                        # Close datasets
                        src_ds = None
                        dst_ds = None

                        # Remove the temporary clipped raster file
                        os.remove(clipped_raster_path)

                    clip_and_remove_zero_pixels(file_path,file_path,polygon_c)

                    selected_bands = [2, 3, 4, 8]
                    with rasterio.open(file_path) as src:
                        read_image = src.read(selected_bands)
                        nir, red = read_image[3], read_image[2]

                    def calculate_ndvi(nir, red):
                        num = nir - red
                        den = nir + red + 0.00000001  # Adding a small value to avoid division by zero
                        ndvi = num / den
                        return ndvi

                    # Calculate NDVI
                    ndvi = calculate_ndvi(nir, red)

                    # Mask out NaN values
                    mask = ~np.isnan(ndvi)
                    non_nan_ndvi = ndvi[mask]

                    # Reshape non-NaN NDVI values
                    n = int(np.sqrt(len(non_nan_ndvi)))
                    m = len(non_nan_ndvi) // n
                    reshaped_ndvi = non_nan_ndvi[:n*m].reshape((n, m))

                    # Calculate VCI (Vegetation Condition Index)
                    ndvi_min = np.nanmin(reshaped_ndvi)
                    ndvi_max = np.nanmax(reshaped_ndvi)

                    if ndvi_max != ndvi_min:
                        vci = 100 * (reshaped_ndvi - ndvi_min) / (ndvi_max - ndvi_min)
                    else:
                        print("Error: Division by zero occurred while calculating VCI. NDVI range is zero.")
                        # Set VCI to a default value
                        vci = np.zeros_like(reshaped_ndvi)

                    print("VCI:", vci)



                    pixel_resolution = 10 * 10

                    m = len(vci)
                    n = len(vci[0])

                    total_pixel = m * n
                    total_area_in_meter_square = total_pixel * pixel_resolution
                    total_area_in_acres1 = total_area_in_meter_square / 4046.86


                    low_stress_mask = np.where((vci > 70) & (vci <= 100), True, False)
                    moderate_stress_mask = np.where((vci > 40) & (vci <= 70), True, False)
                    high_severe_stress_mask = np.where(vci <= 40, True, False)



                    # Area calculation for different stress level masks
                    low_stress_area_m2 = np.sum(low_stress_mask) * pixel_resolution
                    moderate_stress_area_m2 = np.sum(moderate_stress_mask) * pixel_resolution
                    high_severe_stress_area_m2 = np.sum(high_severe_stress_mask) * pixel_resolution

                    low_stress_area_acres = round(low_stress_area_m2 / 4046.86, 2)
                    moderate_stress_area_acres = round(moderate_stress_area_m2 / 4046.86, 2)
                    high_severe_stress_area_acres = round(high_severe_stress_area_m2 / 4046.86, 2)

                    low_stress_area_hectares = round(low_stress_area_m2 / 10000, 2)
                    moderate_stress_area_hectares = round(moderate_stress_area_m2 / 10000, 2)
                    high_severe_stress_area_hectares = round(high_severe_stress_area_m2 / 10000, 2)

                    # Calculate total area
                    total_area_in_acres2 = low_stress_area_acres + moderate_stress_area_acres + high_severe_stress_area_acres
                    total_area_in_hectares = low_stress_area_hectares + moderate_stress_area_hectares + high_severe_stress_area_hectares

                    percentage_stress = round((high_severe_stress_area_acres) / total_area_in_acres2 * 100, 2)
                    k = (f"{round(high_severe_stress_area_acres, 2)} acres ={percentage_stress}% of field under stress")

                    # stress_table = [
                    #     {
                    #         "category": "Low Stress",
                    #         "area_acres": low_stress_area_acres,
                    #         "area_hectares": low_stress_area_hectares
                    #     },
                    #     {
                    #         "category": "Moderate Stress",
                    #         "area_acres": moderate_stress_area_acres,
                    #         "area_hectares": moderate_stress_area_hectares
                    #     },
                    #     {
                    #         "category": "High/Severe Stress",
                    #         "area_acres": high_severe_stress_area_acres,
                    #         "area_hectares": high_severe_stress_area_hectares
                    #     }
                    # ]

                    categories = ['Low Stress', 'Moderate Stress', 'High/Severe Stress']
                    areas = [low_stress_area_acres, moderate_stress_area_acres, high_severe_stress_area_acres]
                    colors = ['#66c2a5', '#fc8d62', '#8da0cb']
                    # colors = ['#4BAF4F', '#f4d35e', '#FF4500']

                    explode = (0.1, 0, 0)
                    wedgeprops = {'linewidth': 2, 'edgecolor': 'white'}
                    shadow = True

                    # Create the pie chart
                    plt.figure(figsize=(8, 8))
                    plt.pie(areas, explode=explode, labels=categories, autopct='%1.1f%%', startangle=140,
                            colors=colors, wedgeprops=wedgeprops, shadow=shadow)

                    plt.title('Stress Levels Area Distribution', fontsize=18, weight='bold')

                    # Customizing legend position
                    legend = plt.legend(categories, loc='lower center', bbox_to_anchor=(0.5, -0.1),
                                        ncol=len(categories), fontsize='medium')

                    # Equal aspect ratio ensures that pie is drawn as a circle
                    plt.axis('equal')
                    # plt.show()
                    pie_chart_rel_path=cc_output_directory+"/plant_stress_pie_chart_"
                    pie_path = f"{pie_chart_rel_path}{save_date}.jpg"
                    plt.savefig(pie_path)
                    plt.close()


                    with rasterio.open(file_path) as image:
                        read_image = image.read()
                        transform_image = image.transform
                        ndvi = calculate_ndvi(nir, red)
                        ndvi_min = np.nanmin(ndvi)
                        ndvi_max = np.nanmax(ndvi)
                        vci = 100 * (ndvi - ndvi_min) / (ndvi_max - ndvi_min)

                        # Masks for different stress levels
                        low_stress_mask = np.where((vci > 50) & (vci <= 100), True, False)
                        moderate_stress_mask = np.where((vci > 20) & (vci <= 50), True, False)
                        high_severe_stress_mask = np.where(vci <= 20, True, False)

                        # Create a mask for NaN values
                        nan_mask = np.isnan(ndvi)

                        # Create a new array for the colored raster
                        colored_raster = np.dstack([
                            np.zeros_like(read_image[0], dtype=np.uint8),
                            np.zeros_like(read_image[0], dtype=np.uint8),
                            np.zeros_like(read_image[0], dtype=np.uint8)
                        ])

                        # Assign different colors for each stress level
                        colored_raster[low_stress_mask] = [0, 128, 0]  # Green for Low Stress
                        colored_raster[moderate_stress_mask] = [255, 255, 0]  # Brown for Moderate Stress
                        colored_raster[high_severe_stress_mask] = [255, 0, 0]  # Red for High/Severe Stress

                        # Set NaN pixels to white
                        colored_raster[nan_mask] = [255, 255, 255]  # White for NaN pixels

                        # Get the extent for plotting
                        extent = (image.bounds.left, image.bounds.right, image.bounds.bottom, image.bounds.top)

                        # Plotting the colored raster
                        fig, ax = plt.subplots(figsize=(10, 10))
                        ax.imshow(colored_raster, extent=extent)

                        plt.title('Visualizing Stress Levels in Raster')

                        # Hide ticks
                        ax.set_xticks([])
                        ax.set_yticks([])

                        # Create custom legend below the image
                        legend_elements = [
                            Patch(facecolor='green', edgecolor='none', label='Low Stress'),
                            Patch(facecolor='yellow', edgecolor='none', label='Moderate Stress'),
                            Patch(facecolor='red', edgecolor='none', label='High/Severe Stress')
                        ]
                        plt.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=3)

                        map_rel_path=cc_output_directory+"/plant_stress_map_"
                        map_path = f"{map_rel_path}{save_date}.jpg"
                        plt.savefig(map_path,dpi=500)
                        plt.close()

                    # date conversion dd-mm-yy
                    # analysis_date=datetime.strptime(analysis_date,'%Y-%m-%d').strftime('%d-%m-%Y')
                    sowing_date=datetime.strptime(sowing_date,'%Y-%m-%d').strftime('%d-%m-%Y')
                    # Data
                    result = {
                        "reportFor":"plant_stress",
                        "service_name": service_name,
                        "crop_name": crop_name,
                        "name": org_name,
                        "sowing_date": sowing_date,
                        "Analysis_date": analysis_date,
                        "crop_area": f"{total_area_in_acres2} acres",
                        "low_stress": low_stress_area_acres,
                        "moderate_stress": moderate_stress_area_acres,
                        "high_stress": high_severe_stress_area_acres,
                        "area under plant stress": k,
                        "pie_chart": f"plant_stress_pie_chart_{save_date}.jpg",
                        "map": f"plant_stress_field_image{save_date}.jpg"
                    }

                    # output jason path
                    output_rel_path=cc_output_directory+"/plant_stress_json_"
                    plant_stress_output_json_path=f"{output_rel_path}{save_date}"

                    # cleared the tiff file from folder

                    with open(plant_stress_output_json_path, 'w') as json_file:
                        json.dump(result, json_file, indent=4)

                    def clear_directory(directory_path):
                        shutil.rmtree(directory_path, ignore_errors=True)
                        os.makedirs(directory_path)

                    directory_path_to_clear = config_data_main["data_output_folder"]

                    clear_directory(directory_path_to_clear)

                    return plant_stress_output_json_path


                except Exception as e:
                    print(f"Error processing file {filename}: {e}")


def main(service_name, crop_name, org_name, analysis_date, polygon_c, sowing_date):

    try:

        # org_name ='abc'
        # crop_name=data['crop_name']
        # sowing_date=data['showing_date']
        # crop_area=128.07
        # service_name=data['service_name']
        # analysis_date=data['analysis_date']
        # polygon_c=data['geom']
        plant_stress_output=plantstress(service_name, crop_name, org_name, analysis_date, polygon_c, sowing_date)
        print(plant_stress_output)

    except:
        print("Service Unavailable at this moment")

if __name__ == "__main__":

    analysis_date = "2023-11-11"
    polygon_c=[[73.09556979663124, 23.8405276975719],
            [73.10241918097321, 23.84029256809925],
            [73.10268407569515, 23.83403065470798],
            [73.0957683354565, 23.83407320230542],
            [73.09556979663124, 23.8405276975719]]
    service_name = "PLANT STRESS ANALYSIS"
    crop_name = "Potato"
    org_name = "Nascent Infotechnologies Pvt. Ltd."
    sowing_date = "2023-11-28"
    # crop_area = 128.07
    # data = json.loads(sys.argv[1])
    try:
        main(service_name,crop_name,org_name,analysis_date,polygon_c,sowing_date)
    except:
        print("internal error")


Error: Division by zero occurred while calculating VCI. NDVI range is zero.
VCI: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


<ipython-input-11-e38bc131d540>:337: RuntimeWarning: divide by zero encountered in divide
  vci = 100 * (ndvi - ndvi_min) / (ndvi_max - ndvi_min)
<ipython-input-11-e38bc131d540>:337: RuntimeWarning: invalid value encountered in divide
  vci = 100 * (ndvi - ndvi_min) / (ndvi_max - ndvi_min)


/root/agri_crop/image/potato/plant_stress/plant_stress_json_2024-03-26-09-03-1711445504


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sentinelhub
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.4/245.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6084 sha256=8c57d836731edd38a6facb700fbab70619c6875104c5f93c9521026e3bc87617
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 44.0 MB/s eta 0:00:00
